In [ ]:
import numpy as np
import pennylane as qml

In [2]:

def U(theta):
    cos_half_theta = np.cos(theta / 2)
    sin_half_theta = np.sin(theta / 2)
    ry_matrix = np.array([
        [cos_half_theta, sin_half_theta],
        [-1*sin_half_theta, cos_half_theta]
    ])
    return ry_matrix


In [3]:
def W_state(start_wire,dimension,psi):
    qml.RY(psi,start_wire)
    qml.PauliX(start_wire+dimension+1)
    qml.CNOT([start_wire,start_wire+dimension+1])
    for i in range (start_wire+dimension+1,start_wire+2*dimension):
        angle=2*np.arccos(np.sqrt(1/(2*dimension-i+1+start_wire)))
        qml.CRY((angle),[i,i+1])
        u=U(angle)
        qml.ControlledQubitUnitary(u, control_wires=[start_wire, i], wires=i+1)
        qml.CNOT([i+1,i])
        qml.MultiControlledX(control_wires=[start_wire,i+1],wires=[i])
    for i in range (start_wire+1,start_wire+dimension+1):
        qml.ctrl(qml.Hadamard,start_wire)(i)
    for i in range (start_wire+dimension+1,start_wire+2*dimension+1):
        qml.MultiControlledX([start_wire,i-dimension],i)

In [4]:
dev_swap = qml.device("lightning.gpu", wires=17)
@qml.qnode(dev_swap)
def Swap_w_state(dimension,psi):
    # for i in range(5*dimension+2):
    #     qml.Hadamard(i)
    for i in range(dimension):
        qml.Hadamard(i)
    W_state(dimension,dimension,psi)
    W_state(3*dimension+1,dimension,psi)
    for i in range (dimension):
        qml.CSWAP([i,i+2*dimension+1,i+4*dimension+2])
    for i in range (dimension):
        qml.Hadamard(i)
    return(qml.probs([i for i in range (dimension)]))

In [5]:
dev_density = qml.device("lightning.gpu", wires=17)
@qml.qnode(dev_density)
def density_w_state(dimension,psi):
    W_state(0,dimension,psi)
    return qml.density_matrix([i for i in range (2*dimension-dimension+1+0,2*dimension+1+0)])

In [7]:
drawer = qml.draw(density_w_state)
print(drawer(3,0))

0: ──RY(0.00)─╭●───────────╭●────────╭●───────────╭●────────╭●─╭●─╭●─╭●─╭●─╭●─╭●─┤       
1: ───────────│────────────│─────────│────────────│─────────│──╰H─│──│──├●─│──│──┤       
2: ───────────│────────────│─────────│────────────│─────────│─────╰H─│──│──├●─│──┤       
3: ───────────│────────────│─────────│────────────│─────────│────────╰H─│──│──├●─┤       
4: ──X────────╰X─╭●────────├●─────╭X─├X───────────│─────────│───────────╰X─│──│──┤ ╭State
5: ──────────────╰RY(1.91)─╰U(M0)─╰●─╰●─╭●────────├●─────╭X─├X─────────────╰X─│──┤ ├State
6: ─────────────────────────────────────╰RY(1.57)─╰U(M1)─╰●─╰●────────────────╰X─┤ ╰State
M0 = 
[[ 0.57735027  0.81649658]
 [-0.81649658  0.57735027]]
M1 = 
[[ 0.70710678  0.70710678]
 [-0.70710678  0.70710678]]


/home/ruibinx/.local/lib/python3.9/site-packages/pennylane/ops/qubit/non_parametric_ops.py:2095: UserWarning: The control_wires keyword will be removed soon. Use wires = (control_wires, target_wire) instead. See the documentation for more information.
  warnings.warn(


In [ ]:
def classical(p, n):
    term1 = ((n - 1) * p**2) / (2 * n)
    term2 = (1 - p**2) * (2 / (2**n) - 1 / (4**n) - (3 / 4)**n)
    result = term1 + term2
    return result
def CE(n,i):
    w=density_w_state(n,i)
    P=np.trace(np.dot(w,w))
    sum_=Swap_w_state(n,i)[0]
    return(1/2**n+(1-1/2**n)*P-sum_)